In [ ]:
import sys
import os
# Add the parent directory to the PYTHONPATH
sys.path.append(os.path.abspath('../'))

import numpy as np
import helper as hp
import pandas as pd
import multiprocessing as mp
from configparser import ConfigParser
from evostrat.init_mlp import MLP
from evostrat.evolution_strategy import EvolutionStrategy
from kinetics.jacobian_solver import check_jacobian


In [ ]:
#Parse arguments from configfile
configs = ConfigParser()
configs.read('configfile.ini')

n_samples = int(configs['MLP']['n_samples'])

lnminkm = float(configs['CONSTRAINTS']['min_km'])
lnmaxkm = float(configs['CONSTRAINTS']['max_km'])

repeats = int(configs['EVOSTRAT']['repeats'])
save_step = int(configs['EVOSTRAT']['save_step'])
generations = int(configs['EVOSTRAT']['generations'])
pop_size = int(configs['EVOSTRAT']['pop_size'])
noise = float(configs['EVOSTRAT']['noise'])
lr = float(configs['EVOSTRAT']['lr'])
decay = float(configs['EVOSTRAT']['decay'])
ss_idx = int(configs['EVOSTRAT']['ss_idx'])
n_threads = int(configs['EVOSTRAT']['n_threads'])

base = configs['PATHS']['base']
output_path = configs['PATHS']['output_path']
met_model = configs['PATHS']['met_model']
model_file = configs['PATHS']['model_file']
thermo_experiment_file = configs['PATHS']['thermo_experiment_file']
regulation_file = configs['PATHS']['regulation_file']
kinetic_params_file = configs['PATHS']['kinetic_params_file']
steady_states_file = configs['PATHS']['steady_states_file']

reward_flag = int(configs['REWARDS']['reward_flag'])
eig_partition = float(configs['REWARDS']['eig_partition'])
n_consider = int(configs['REWARDS']['n_consider'])

pf_flag = int(configs['PARAMETER_FIXING']['pf_flag'])


In [ ]:
path_to_kmodel = f'{base}/{met_model}/kinetic/{model_file}'
path_to_tmodel = f'{base}/{met_model}/thermo/{thermo_experiment_file}'
path_to_parameters = f'{base}/{met_model}/parameters/{kinetic_params_file}'
path_to_regulation = f'{base}/{met_model}/{regulation_file}'
path_to_samples = f'{base}/{met_model}/steady_state_samples/{steady_states_file}'

In [ ]:
## Load kinetic and thermodynamic data with regulation  
regulation = pd.read_csv(path_to_regulation, header=0)
# Call solvers from SKimPy
chk_jcbn = check_jacobian()
# Integrate regulation data
chk_jcbn.load_regulation_models(path_to_kmodel, path_to_tmodel, path_to_samples, regulation, 1712) 

In [ ]:
# Load steady states data into the model
steady_states = pd.read_csv(path_to_samples, header=0, index_col=0).iloc[ss_idx, 0:]
chk_jcbn._load_ssprofile(steady_states)  ## Integrate steady state information

In [ ]:
k_names = []
k_regulation = []

for k, v in chk_jcbn.kmodel.parameters.items():
    if k.startswith("km_") or "activator" in k or "inhibitor" in k or "activation" in k or "inhibition" in k:
        if k not in k_names:
            k_names.append(k)
            
        if "activator" in k or "inhibitor" in k or "activation" in k or "inhibition" in k:
            k_regulation.append(k)

print(f"The number of k for generation is {len(k_names)}, \nand the number of regulation parameters is {len(k_regulation)}.")

In [ ]:
parameter_sample= {v.symbol: v.value for k,v in chk_jcbn.kmodel.parameters.items()}

print(parameter_sample)

In [ ]:
# Create a DataFrame from list1
df = pd.DataFrame(chk_jcbn.kmodel.parameters.keys(), columns=['Full List'])

# Initialize the second column with NaNs or an appropriate placeholder
df['Selected'] = pd.Series([None] * len(df))

# Populate the 'Selected' column with values from list2 where they match values in 'Full List'
for item in k_names:
    if item in df['Full List'].values:
        df.loc[df['Full List'] == item, 'Selected'] = item

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('matched_list.csv', index=False)

In [ ]:
# declare reward functions
def reward_func(weights):
    """
    evaluate reward for a set of generated kinetic parameter sets
    :param gen_params: agent generated kinetic parameter sets
    :return: reward
    """

    global calc_eig
    global n_samples
    global eig_partition
    global n_threads
    global reward_flag
    global gen_params
    global n_consider

    def calc_eig(gen_param):
        chk_jcbn._prepare_parameters(gen_param, k_names)
        max_eig = chk_jcbn.calc_eigenvalues_recal_vmax()
        return max_eig

    pool = mp.Pool(n_threads)

    mlp.generator.set_weights(weights)
    gen_params = mlp.sample_parameters()
    gen_params = [[params] for params in gen_params]

    max_eig = pool.map(calc_eig, gen_params)
    # for gen_param in gen_params:
    #     calc_eig(gen_param)
    # max_eig = np.array([this_eig for eig in max_eig for this_eig in eig])

    if reward_flag == 0:
        max_neg_eig = np.min(max_eig)
        if max_neg_eig > eig_partition:
            this_reward = 0.01 / (1 + np.exp(max_neg_eig - eig_partition))
        else:
            this_reward = len(np.where(max_eig <= eig_partition)[0]) / n_samples
        pool.close()
    elif reward_flag == 1:
        max_eig.sort()
        considered_avg = sum(max_eig[:n_consider]) / n_consider
        this_reward = np.exp(-0.1 * considered_avg) / 2
        pool.close()
    return this_reward

In [ ]:

for rep in range(1):
    print(f"repeat {rep}")
    cond_class = 1
    # Call neural network agent
    mlp = MLP(cond_class, lnminkm, lnmaxkm, n_samples, k_names, len(k_names), param_fixing=pf_flag)
    '''
    ## Snippet to start from previously saved weights 
    load_weights = hp.load_pkl('/path/to/weights')
    mlp.generator.set_weights(load_weights)
    '''
    init_dict = mlp.generator.get_weights()

    this_savepath = f'../{output_path}/repeat_{rep}/'
    os.makedirs(this_savepath, exist_ok=True)

    es = EvolutionStrategy(mlp.generator.get_weights(),
                           reward_func, this_savepath,
                           population_size=pop_size,
                           sigma=noise,  # noise std deviation
                           learning_rate=lr,
                           decay=decay,
                           num_threads=1)
    
    rewards = es.run(generations, print_step=save_step)
    hp.save_pkl(f'../{this_savepath}/rewards', rewards)



In [ ]:
chk_jcbn.conc_series

In [ ]:
chk_jcbn.parameter_sample_set[0]

In [ ]:
print(chk_jcbn.jacobian_set[0])
print("Shape:", chk_jcbn.jacobian_set[20].shape)
print("Non-zero elements:", chk_jcbn.jacobian_set[20].nnz)



In [ ]:
len(chk_jcbn.jacobian_set)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix

# Creating a sample matrix with some NaN values
matrix = chk_jcbn.jacobian_set[0]
dense_matrix = matrix.toarray()

# Use a colormap that represents NaN values with grey
# 'viridis' is just an example; you can use any suitable colormap
cmap = plt.cm.viridis
cmap.set_bad(color='grey')

# Create the heatmap
plt.figure(figsize=(8, 6))
plt.imshow(dense_matrix, interpolation='nearest', cmap=cmap)

# Add color bar for reference
plt.colorbar()

# Add title and labels as needed
plt.title('Heatmap of Matrix')
plt.xlabel('Column')
plt.ylabel('Row')

# Show the plot
plt.show()

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csc_matrix

# Create the output directory if it doesn't exist
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Assuming sparse_matrices is your list of csc_matrix objects
# sparse_matrices = [csc_matrix1, csc_matrix2, ..., csc_matrix100]

for i, sparse_matrix in enumerate(chk_jcbn.jacobian_set):
    # Convert the sparse matrix to a dense array
    dense_matrix = sparse_matrix.toarray()
    
    # Use a colormap and set NaN values to appear grey
    cmap = plt.cm.viridis
    cmap.set_bad(color='grey')
    
    # Create the heatmap
    plt.figure(figsize=(8, 6))
    plt.imshow(dense_matrix, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    plt.title(f'Heatmap {i+1}')
    plt.xlabel('Column')
    plt.ylabel('Row')
    
    # Save the plot to the output directory with a unique name
    plt.savefig(f'{output_dir}/heatmap_{i+1}.png')
    plt.close()  # Close the plot to free up memory

In [ ]:
type(chk_jcbn.jacobian_set[0])

In [ ]:
chk_jcbn.kmodel.jacobian_fun

## Save parameters

In [ ]:
set_0 = gen_params[0][0]
gen_flat = [inner_list[0] for inner_list in gen_params]


In [ ]:
# Convert the numpy array to a DataFrame
df = pd.DataFrame(gen_flat, columns=k_names)

# Save the DataFrame to a CSV file
df.to_csv('gen_parms.csv', index=False)